# Enhanced LSD Line Matching with Multi-Scale and Color Augmentation

This notebook demonstrates an enhanced approach to line segment detection and matching using LSD (Line Segment Detector) and Binary Descriptors in OpenCV.
It incorporates several improvements over basic LSD matching:
- **Multi-Scale Template Processing**: The template image is processed at multiple scales (0.7x, 1.0x, 1.3x) to handle variations in object size.
- **Color Augmentation**: HSV color histograms are extracted along detected line segments and appended to their binary descriptors, providing richer feature information.
- **Geometric Filtering**: Matches are refined using geometric consistency checks, including angle similarity and line length ratios.
- **Descriptor Optimization**: LSD detection uses 3 octaves, and Binary Descriptors are computed with a wider band width (9px) for more contextual information.
- **Robust Matching**: An adaptive approach to find the best matches by considering scores from different template scales and applying geometric verification.

This notebook adapts logic from the original `lsd_matching.ipynb` and incorporates concepts from the provided reference Python script.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# --- Parameters ---

# Image Paths
template_base_folder = "/Users/frbuccoliero/Desktop/Thesis/Tests/utils/wikiCommonsOutput/png/"
reference_image_path = "/Users/frbuccoliero/Desktop/Thesis/Tests/template_matching/dataset/images/positive/positive_00000.jpg"
template_image_path = None # Will be selected from the folder

# Multi-Scale Processing
TEMPLATE_SCALES = [0.7, 1.0, 1.3]

# LSD Detector Parameters
LSD_INTERNAL_SCALE_FACTOR = 2  # Internal scale factor for LSD algorithm
LSD_NUM_OCTAVES = 3          # Number of octaves for multi-octave detection

# Binary Descriptor Parameters
BD_BAND_WIDTH = 9            # Width of the band for descriptor computation

# Matching and Filtering Parameters
MATCHES_DIST_THRESHOLD = 50.0 # Initial Hamming distance threshold for potential matches
ANGLE_TOLERANCE_DEG = 15.0    # Angle tolerance for geometric filtering (degrees)
LENGTH_RATIO_MIN = 0.7        # Minimum length ratio for geometric filtering
LENGTH_RATIO_MAX = 1.3        # Maximum length ratio for geometric filtering

# Visualization
MAX_MATCHES_TO_DISPLAY = 50   # Max matches to show in plots

# Select a template image (e.g., the 9th one as in the original notebook)
if os.path.exists(template_base_folder) and os.path.isdir(template_base_folder):
    potential_templates = sorted([
        os.path.join(template_base_folder, f)
        for f in os.listdir(template_base_folder)
        if f.lower().endswith(".png")
    ])
    if potential_templates and len(potential_templates) > 8:
        template_image_path = potential_templates[8]
        print(f"Using template image: {template_image_path}")
    elif potential_templates:
        template_image_path = potential_templates[0] # Fallback to the first one
        print(f"Fewer than 9 templates, using first one: {template_image_path}")
    else:
        print(f"No PNG template images found in {template_base_folder}. Please set 'template_image_path' manually.")
else:
    print(f"Template folder '{template_base_folder}' does not exist. Please set 'template_image_path' manually.")


In [ ]:
# --- Load Images ---

img1_orig = None  # Template
img2_orig = None  # Reference
img1_gray = None
img2_gray = None

if template_image_path and os.path.exists(template_image_path):
    img1_orig = cv2.imread(template_image_path)
    if img1_orig is not None:
        img1_gray = cv2.cvtColor(img1_orig, cv2.COLOR_BGR2GRAY)
        print(f"Template image '{os.path.basename(template_image_path)}' loaded successfully.")
    else:
        print(f"Error loading template image: {template_image_path}")
else:
    print(f"Template image path is not valid or not set: {template_image_path}")

if os.path.exists(reference_image_path):
    img2_orig = cv2.imread(reference_image_path)
    if img2_orig is not None:
        img2_gray = cv2.cvtColor(img2_orig, cv2.COLOR_BGR2GRAY)
        print(f"Reference image '{os.path.basename(reference_image_path)}' loaded successfully.")
    else:
        print(f"Error loading reference image: {reference_image_path}")
else:
    print(f"Reference image path does not exist: {reference_image_path}")

if img1_gray is None or img2_gray is None:
    print("One or both images could not be loaded. Halting execution.")
    # In a real script, you might raise an error or exit here.
else:
    plt.figure(figsize=(12, 6))
    plt.subplot(121)
    plt.imshow(cv2.cvtColor(img1_orig, cv2.COLOR_BGR2RGB))
    plt.title(f"Template: {os.path.basename(template_image_path)}")
    plt.axis('off')
    plt.subplot(122)
    plt.imshow(cv2.cvtColor(img2_orig, cv2.COLOR_BGR2RGB))
    plt.title(f"Reference: {os.path.basename(reference_image_path)}")
    plt.axis('off')
    plt.show()

In [ ]:
# --- Helper Functions ---

def rescale_image(image, scale):
    """Rescales an image by a given factor."""
    if scale == 1.0:
        return image.copy()
    width = int(image.shape[1] * scale)
    height = int(image.shape[0] * scale)
    dim = (width, height)
    return cv2.resize(image, dim, interpolation=cv2.INTER_AREA)

def get_line_points(keyline, num_samples=10):
    """Generates sample points along a line segment (KeyLine)."""
    x0, y0 = keyline.startPointX, keyline.startPointY
    x1, y1 = keyline.endPointX, keyline.endPointY
    
    xs = np.linspace(x0, x1, num_samples, dtype=int)
    ys = np.linspace(y0, y1, num_samples, dtype=int)
    
    # Ensure points are within image bounds (caller should handle this if image is passed)
    # For simplicity, we just return the points here.
    return np.vstack((xs, ys)).T

def compute_color_descriptors(image_bgr, keylines, num_hist_bins=8):
    """Computes HSV color histograms along line segments."""
    if not keylines or len(keylines) == 0:
        return np.array([], dtype=np.float32).reshape(0, num_hist_bins)

    hsv_image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2HSV)
    h, w = hsv_image.shape[:2]
    descriptors = []

    for kl in keylines:
        points = get_line_points(kl, num_samples=10)
        
        # Clip points to be within image boundaries
        points[:, 0] = np.clip(points[:, 0], 0, w - 1)
        points[:, 1] = np.clip(points[:, 1], 0, h - 1)
        
        # Sample hues
        hues = hsv_image[points[:, 1], points[:, 0], 0] # H channel
        
        # Compute histogram
        hist = cv2.calcHist([hues], [0], None, [num_hist_bins], [0, 180])
        cv2.normalize(hist, hist, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX) # Normalize
        descriptors.append(hist.flatten())
        
    return np.array(descriptors, dtype=np.float32)

def filter_matches_geometrically(matches, query_keylines, template_keylines, template_scale,
                                 angle_tolerance_deg, length_ratio_min, length_ratio_max):
    """Filters matches based on geometric consistency (angle and length ratio)."""
    if not matches:
        return []
    
    valid_matches = []
    angle_tolerance_rad = np.deg2rad(angle_tolerance_deg)

    for m in matches:
        q_kl = query_keylines[m.queryIdx]
        t_kl = template_keylines[m.trainIdx]

        # 1. Angle consistency
        # OpenCV KeyLine angle is in radians, measured from x-axis.
        # Ensure angles are in [0, pi] or handle periodicity if they can be [-pi, pi]
        # LSD angles are typically [0, 2pi) or similar. Let's assume they might need normalization for diff.
        angle_q = q_kl.angle
        angle_t = t_kl.angle
        
        # Normalize angle difference to be within [-pi, pi]
        angle_diff = angle_q - angle_t
        while angle_diff > np.pi:
            angle_diff -= 2 * np.pi
        while angle_diff < -np.pi:
            angle_diff += 2 * np.pi
        
        if abs(angle_diff) > angle_tolerance_rad:
            continue

        # 2. Length ratio validation
        # q_len is length in query image (original scale)
        # t_len_scaled is length in template image (which was scaled by template_scale)
        # Expected q_len approx. (t_len_scaled / template_scale)
        # Ratio: q_len / (t_len_scaled / template_scale) = (q_len * template_scale) / t_len_scaled
        q_len = q_kl.lineLength
        t_len_scaled = t_kl.lineLength

        if t_len_scaled < 1e-3: # Avoid division by zero or tiny length
            continue
            
        effective_length_ratio = (q_len * template_scale) / t_len_scaled
        
        if not (length_ratio_min <= effective_length_ratio <= length_ratio_max):
            continue
            
        valid_matches.append(m)
    return valid_matches


In [ ]:
# --- Initialize Detectors and Matcher ---

# LSD Line Segment Detector
lsd = cv2.line_descriptor.LSDDetector.createLSDDetector()

# Binary Descriptor Extractor
bd = cv2.line_descriptor.BinaryDescriptor.createBinaryDescriptor()
bd.setWidthOfBand(BD_BAND_WIDTH)

# Binary Descriptor Matcher
bdm = cv2.line_descriptor.BinaryDescriptorMatcher()

print("LSD Detector, Binary Descriptor (with wider band), and Matcher initialized.")

In [ ]:
# --- Preprocess Template Image (Multi-Scale with Color Augmentation) ---

processed_templates_data = []

if img1_orig is not None:
    print("Processing template image at multiple scales...")
    for scale_factor in TEMPLATE_SCALES:
        print(f"  Processing scale: {scale_factor}x")
        
        # Rescale template image
        template_scaled_bgr = rescale_image(img1_orig, scale_factor)
        template_scaled_gray = cv2.cvtColor(template_scaled_bgr, cv2.COLOR_BGR2GRAY)
        
        # Detect lines using LSD
        # Note: LSD's `scale` param is different from our image `scale_factor`.
        # It refers to the image pyramid scale within LSD.
        keylines_raw_template = lsd.detect(template_scaled_gray, 
                                           scale=LSD_INTERNAL_SCALE_FACTOR, 
                                           numOctaves=LSD_NUM_OCTAVES)
        
        if not keylines_raw_template or len(keylines_raw_template) == 0:
            print(f"    No raw lines detected for scale {scale_factor}x.")
            processed_templates_data.append({
                'scale': scale_factor,
                'image_bgr': template_scaled_bgr,
                'keylines': [],
                'descriptors_enhanced': np.array([], dtype=np.float32)
            })
            continue

        # Compute binary descriptors
        # OpenCV's BinaryDescriptor.compute expects the image where keylines were found.
        # The reference code passes the color image. Let's try that.
        # If it fails, it might need template_scaled_gray.
        # However, color descriptors are computed separately from the color image.
        keylines_template, descs_template_binary = bd.compute(template_scaled_bgr, keylines_raw_template)
        
        if descs_template_binary is None or len(descs_template_binary) == 0:
            print(f"    No binary descriptors computed for scale {scale_factor}x.")
            processed_templates_data.append({
                'scale': scale_factor,
                'image_bgr': template_scaled_bgr,
                'keylines': keylines_template if keylines_template is not None else [],
                'descriptors_enhanced': np.array([], dtype=np.float32)
            })
            continue
            
        print(f"    Scale {scale_factor}x: Detected {len(keylines_raw_template)} raw lines. Computed {len(descs_template_binary)} binary descriptors for {len(keylines_template)} lines.")

        # Compute color descriptors
        descs_template_color = compute_color_descriptors(template_scaled_bgr, keylines_template)
        print(f"    Scale {scale_factor}x: Computed {len(descs_template_color)} color descriptors.")

        # Augment descriptors (hstack requires float type for binary descriptors)
        descs_template_enhanced = np.hstack((descs_template_binary.astype(np.float32), descs_template_color))
        
        processed_templates_data.append({
            'scale': scale_factor,
            'image_bgr': template_scaled_bgr, # Store scaled BGR image for visualization
            'keylines': keylines_template,
            'descriptors_enhanced': descs_template_enhanced
        })
    print("Finished processing template scales.")
else:
    print("Template image (img1_orig) not loaded. Skipping template processing.")


In [ ]:
# --- Process Reference Image (with Color Augmentation) ---

keylines_ref = None
descs_ref_enhanced = None

if img2_gray is not None and img2_orig is not None:
    print("Processing reference image...")
    # Detect lines using LSD
    keylines_raw_ref = lsd.detect(img2_gray, 
                                  scale=LSD_INTERNAL_SCALE_FACTOR, 
                                  numOctaves=LSD_NUM_OCTAVES)

    if keylines_raw_ref and len(keylines_raw_ref) > 0:
        # Compute binary descriptors
        keylines_ref, descs_ref_binary = bd.compute(img2_orig, keylines_raw_ref) # Using color image as per template processing

        if descs_ref_binary is not None and len(descs_ref_binary) > 0:
            print(f"  Reference: Detected {len(keylines_raw_ref)} raw lines. Computed {len(descs_ref_binary)} binary descriptors for {len(keylines_ref)} lines.")
            
            # Compute color descriptors
            descs_ref_color = compute_color_descriptors(img2_orig, keylines_ref)
            print(f"  Reference: Computed {len(descs_ref_color)} color descriptors.")

            # Augment descriptors
            descs_ref_enhanced = np.hstack((descs_ref_binary.astype(np.float32), descs_ref_color))
            print(f"  Reference: Enhanced descriptors shape: {descs_ref_enhanced.shape}")
        else:
            print("  Reference: No binary descriptors computed.")
            keylines_ref = keylines_raw_ref # Keep raw keylines if no descriptors
    else:
        print("  Reference: No raw lines detected.")
    print("Finished processing reference image.")
else:
    print("Reference image (img2_gray or img2_orig) not loaded. Skipping reference processing.")


In [ ]:
# --- (Optional) Plot Detected Lines ---
# This cell shows lines on the 1.0x scaled template and the reference image.

if processed_templates_data and keylines_ref:
    # Find the 1.0x scaled template data
    template_1x_data = next((t for t in processed_templates_data if t['scale'] == 1.0), None)

    fig, axes = plt.subplots(1, 2, figsize=(20, 10))
    
    # Plot lines on the 1.0x template
    if template_1x_data and template_1x_data['keylines']:
        ax1 = axes[0]
        img_to_show_template = cv2.cvtColor(template_1x_data['image_bgr'], cv2.COLOR_BGR2RGB)
        ax1.imshow(img_to_show_template)
        ax1.set_title(f'Template (1.0x scale) with Detected Lines ({len(template_1x_data["keylines"])})')
        ax1.axis('off')
        for kl in template_1x_data['keylines']:
            pt1 = (kl.startPointX, kl.startPointY)
            pt2 = (kl.endPointX, kl.endPointY)
            ax1.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='lime', linewidth=1)
    else:
        axes[0].set_title('Template (1.0x scale) - No lines or data')
        axes[0].axis('off')

    # Plot lines on the reference image
    ax2 = axes[1]
    img_to_show_ref = cv2.cvtColor(img2_orig, cv2.COLOR_BGR2RGB)
    ax2.imshow(img_to_show_ref)
    ax2.set_title(f'Reference Image with Detected Lines ({len(keylines_ref)})')
    ax2.axis('off')
    if keylines_ref:
        for kl in keylines_ref:
            pt1 = (kl.startPointX, kl.startPointY)
            pt2 = (kl.endPointX, kl.endPointY)
            ax2.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='lime', linewidth=1)
            
    plt.tight_layout()
    plt.show()
else:
    print("Not enough data to plot detected lines (template or reference lines missing).")

In [ ]:
# --- Matching, Filtering, and Scoring ---

all_match_results = []

if descs_ref_enhanced is not None and len(descs_ref_enhanced) > 0 and processed_templates_data:
    print("Starting matching process across template scales...")
    for template_data in processed_templates_data:
        current_scale = template_data['scale']
        descs_template_current = template_data['descriptors_enhanced']
        keylines_template_current = template_data['keylines']
        
        if descs_template_current is None or len(descs_template_current) == 0 or \
           keylines_template_current is None or len(keylines_template_current) == 0:
            print(f"  Skipping scale {current_scale}x: No descriptors or keylines for template.")
            continue

        print(f"  Matching against template scale: {current_scale}x")
        
        # 1. Raw matching using BDM (Binary Descriptor Matcher)
        # Query descriptors first (reference), then train descriptors (template)
        raw_matches = bdm.match(descs_ref_enhanced, descs_template_current)
        print(f"    Scale {current_scale}x: Found {len(raw_matches)} raw matches.")

        # 2. Filter by initial Hamming distance (optional, can be generous)
        # This step is similar to the original notebook's "good_matches"
        # It can help reduce the number of matches before more expensive geometric filtering.
        distance_filtered_matches = [m for m in raw_matches if m.distance < MATCHES_DIST_THRESHOLD]
        print(f"    Scale {current_scale}x: {len(distance_filtered_matches)} matches after distance threshold (< {MATCHES_DIST_THRESHOLD}).")
        
        # 3. Geometric Filtering
        geometrically_filtered_matches = filter_matches_geometrically(
            distance_filtered_matches,
            keylines_ref,
            keylines_template_current,
            current_scale, # Pass the template's current scale factor
            ANGLE_TOLERANCE_DEG,
            LENGTH_RATIO_MIN,
            LENGTH_RATIO_MAX
        )
        print(f"    Scale {current_scale}x: {len(geometrically_filtered_matches)} matches after geometric filtering.")

        # 4. Calculate score (e.g., number of good matches, possibly weighted by scale)
        # Reference code uses: len(valid_matches) * max(0.5, 1/scale)
        # This gives higher scores to smaller scale templates if they achieve same number of matches.
        score = len(geometrically_filtered_matches) * max(0.5, 1.0 / current_scale if current_scale > 1e-6 else 1.0)
        
        all_match_results.append({
            'scale': current_scale,
            'matches': geometrically_filtered_matches,
            'score': score,
            'template_keylines': keylines_template_current,
            'template_image_bgr': template_data['image_bgr'] # For drawing
        })
        
    # Sort results by score in descending order
    if all_match_results:
        all_match_results.sort(key=lambda x: x['score'], reverse=True)
        print("\nMatching results sorted by score:")
        for res in all_match_results:
            print(f"  Scale {res['scale']}x: Score = {res['score']:.2f}, Matches = {len(res['matches'])}")
    else:
        print("No match results obtained.")
        
else:
    print("Cannot proceed with matching: Reference descriptors or processed template data is missing.")


In [ ]:
# --- Display Best Match ---

if all_match_results:
    best_result = all_match_results[0]
    print(f"\nBest result from scale: {best_result['scale']}x with score: {best_result['score']:.2f} and {len(best_result['matches'])} matches.")

    # Prepare for drawing
    best_matches_to_draw = best_result['matches'][:MAX_MATCHES_TO_DISPLAY]
    
    # Reference image (query)
    img_ref_draw = img2_orig.copy() 
    # Template image at the best scale
    img_template_draw = best_result['template_image_bgr'].copy() 

    keylines_ref_for_draw = keylines_ref
    keylines_template_for_draw = best_result['template_keylines']

    # Create a combined image to draw matches
    h1, w1 = img_template_draw.shape[:2]
    h2, w2 = img_ref_draw.shape[:2]
    
    # Create a new canvas for combined image
    # Ensure consistent height for side-by-side display
    max_height = max(h1, h2)
    # Resize images to have same height if needed, maintaining aspect ratio
    if h1 != max_height:
        img_template_draw = cv2.resize(img_template_draw, (int(w1 * max_height / h1), max_height))
        w1 = img_template_draw.shape[1] # Update width
    if h2 != max_height:
        img_ref_draw = cv2.resize(img_ref_draw, (int(w2 * max_height / h2), max_height))
        # w2 is not needed for offset if query is on the left

    # Combined image: Reference on left, Template on right
    # This is different from original notebook's template|reference. Let's do template|reference.
    # Template on left (img_template_draw), Reference on right (img_ref_draw)
    
    # Update dimensions after potential resize
    h1, w1 = img_template_draw.shape[:2]
    h2, w2 = img_ref_draw.shape[:2]
    
    # Ensure both images have the same height for clean concatenation
    target_h = max(h1, h2)
    if h1 < target_h:
        img_template_draw = cv2.resize(img_template_draw, (int(w1 * target_h / h1), target_h))
        w1 = img_template_draw.shape[1]
    if h2 < target_h:
        img_ref_draw = cv2.resize(img_ref_draw, (int(w2 * target_h / h2), target_h))
        w2 = img_ref_draw.shape[1]


    combined_img_width = w1 + w2
    combined_img = np.zeros((target_h, combined_img_width, 3), dtype=np.uint8)
    combined_img[:target_h, :w1] = img_template_draw
    combined_img[:target_h, w1:w1+w2] = img_ref_draw

    # Draw lines and connections
    for match in best_matches_to_draw:
        # queryIdx refers to reference image descriptors, trainIdx to template image descriptors
        idx_ref = match.queryIdx 
        idx_template = match.trainIdx

        if idx_template < len(keylines_template_for_draw) and idx_ref < len(keylines_ref_for_draw):
            kl_template = keylines_template_for_draw[idx_template]
            kl_ref = keylines_ref_for_draw[idx_ref]

            # Points for template line (on the left part of combined_img)
            pt1_template = (int(kl_template.startPointX), int(kl_template.startPointY))
            pt2_template = (int(kl_template.endPointX), int(kl_template.endPointY))
            cv2.line(combined_img, pt1_template, pt2_template, (0, 255, 255), 1) # Cyan for template lines

            # Points for reference line (on the right part, so X needs offset w1)
            pt1_ref = (int(kl_ref.startPointX) + w1, int(kl_ref.startPointY))
            pt2_ref = (int(kl_ref.endPointX) + w1, int(kl_ref.endPointY))
            cv2.line(combined_img, pt1_ref, pt2_ref, (0, 255, 0), 1) # Lime for reference lines
            
            # Draw connecting line between start points (or midpoints)
            # Using start points for this example
            cv2.line(combined_img, pt1_template, pt1_ref, (255, 0, 0), 1) # Red connecting lines
    
    plt.figure(figsize=(20, 10))
    plt.imshow(cv2.cvtColor(combined_img, cv2.COLOR_BGR2RGB))
    title_str = (f"Best Matches (Scale {best_result['scale']}x, Score {best_result['score']:.2f}) - "
                 f"Showing {len(best_matches_to_draw)} of {len(best_result['matches'])} geom. filtered matches")
    plt.title(title_str)
    plt.axis('off')
    plt.show()

elif processed_templates_data and descs_ref_enhanced is not None:
    print("Matching process completed, but no suitable matches found across any scale after filtering.")
else:
    print("Best match display skipped: No results from matching or prerequisites missing.")


In [ ]:
# --- End of Notebook ---
# Further analysis or parameter tuning can be done based on these results.
# For example, exploring different scoring mechanisms, color descriptor variations,
# or more advanced geometric verification (e.g., RANSAC-based homography).